In [2]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [3]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [4]:
stats = pd.read_csv(path + '/all_files_stats.csv')
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [5]:
listcol= files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
listcol.filename = listcol.filename.str.split('_', expand=True)[0]
listcol.index = listcol['filename']
listcol

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [148]:
listcolt = listcol#[listcol['filename']=='Recipe']
listcolt

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [124]:
def process(profiles):
    #print(filename)
    df = pd.read_json(os.path.join(inputdir+profiles), compression='gzip', lines=True)
    #if len(df)<=50:
        #print(len(df))
    #    return
    under50 = set(listProd50)
    setfile = set(df.columns)
    length50 = len(setfile & under50)
    length = len(setfile & setcol)
    #print(length)
    if (length50 > 0):
        if (length >= 3): #& (length<= 12):
            df.to_json(os.path.join(outputdir+profiles), compression='gzip', orient='records', lines=True)

In [125]:
if __name__ == '__main__':
    for index in listcolt.filename:
        print(index)
        setcol = set(listcolt.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/data/'+index+'/All/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/under50/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(process, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(process, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        p.join()
        

Event
{'doortime', 'eventstatus', 'performer', 'name', 'duration', 'location', 'typicalagerange', 'enddate', 'performers', 'eventattendancemode', 'organizer'}


100%|██████████| 22464/22464 [00:22<00:00, 1010.12it/s]


159


In [149]:
num_tables=[]
for index in listcolt.filename:
    print(index)
    outputdir = '../../src/data/schemafiltereddata/'+index+'/'
    print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
    num_tables.append(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
len(num_tables)

Book
978
CreativeWork
5518
Event
6940
Hotel
1001
LocalBusiness
4252
MusicAlbum
248
MusicRecording
12661
Person
4436
Place
1305
Product
33203
Recipe
7832
Restaurant
734
TVEpisode
210


13

In [156]:
#listcol['numtables']=num_tables
listcol['numtables'].sum()
listcol
listcol.to_csv('../../src/data/data/CSV_files/stats_filtered_and_cleaned_tables.csv', index_label=False)

In [144]:
column_count_Music = []
path='../../src/data/schemafiltereddata/'+'MusicRecording'+'/'
files = [file for file in os.listdir(path) if file.endswith('.json.gz')]
for file in files:
    # read file
    df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
    list_col = list(df.columns)
    column_count_Music.extend(list_col)

In [157]:
listcolt = pd.read_csv('../../src/data/data/CSV_files/stats_filtered_and_cleaned_tables.csv')
listcolt = listcolt.set_index('filename')
listcolt

,listofcolumns,numtables
filename,,
Book,"['name', 'author', 'isbn', 'publisher', 'offer...",978
CreativeWork,"['headline', 'author', 'datepublished', 'datem...",5518
Event,"['name', 'enddate', 'location', 'performer', '...",6940
Hotel,"['name', 'address', 'telephone', 'pricerange',...",1001
LocalBusiness,"['name', 'address', 'telephone', 'geo', 'price...",4252
MusicAlbum,"['name', 'byartist', 'track', 'numtracks']",248
MusicRecording,"['name', 'duration', 'byartist', 'inalbum', 'd...",12661
Person,"['mainentityofpage', 'jobtitle', 'birthdate', ...",4436
Place,"['name', 'geo', 'telephone', 'sameas', 'openin...",1305


In [145]:
import ast
MusicR_list = listcolt.loc['MusicRecording']['listofcolumns']
MusicR_list = ast.literal_eval(MusicR_list)
#MusicR_list

In [146]:
colMusicR = pd.DataFrame()
colMusicR['colcount_MusicR'] = column_count_Music
colMusicR = colMusicR.groupby('colcount_MusicR').agg({'colcount_MusicR':'count'})
colMusicR = colMusicR.rename(columns={'colcount_MusicR':'num_col'}).sort_values(by='num_col').reset_index()
colMusicR = colMusicR[colMusicR['colcount_MusicR'].isin(MusicR_list)]
colMusicR
#Prod100 = colMusicR[colMusicR['num_col']<=50]
#Prod100['agg']=1
#listProd50 = Prod100.groupby('agg')['colcount_MusicR'].apply(list).reset_index()
#listProd50 = listProd50.loc[0,'colcount_MusicR']

,colcount_MusicR,num_col
71,datecreated,149
72,genre,165
73,datepublished,225
75,inalbum,9192
76,byartist,11922
77,duration,12420
78,name,12650


In [147]:
colProduct.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsProduct.csv')
colLocalB.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsLocalBusiness.csv')
colCreativeW.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsCreativeWork.csv')
colEvent.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsEvent.csv')
colRecipe.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsRecipe.csv')
colMusicR.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsMusicRecording.csv')
display(colMusicR)
display(colRecipe)
display(colCreativeW)
display(colProduct)
display(colLocalB)
display(colEvent)

,colcount_MusicR,num_col
71,datecreated,149
72,genre,165
73,datepublished,225
75,inalbum,9192
76,byartist,11922
77,duration,12420
78,name,12650


,colcount_Recipe,num_col
101,headline,125
102,publisher,165
103,datemodified,193
104,suitablefordiet,316
105,review,316
106,cookingmethod,337
109,performtime,635
111,ingredients,1611
112,ispartof,2804
113,mainentityofpage,2839


,colcount_CreativeW,num_col
103,copyrightholder,68
104,copyrightyear,73
106,interactioncount,96
107,creator,99
108,inlanguage,112
109,version,121
110,offers,127
111,aggregaterating,129
112,keywords,135
113,genre,164


,colcount_Product,num_col
327,offerdetails,20
346,reviews,65
347,itemlistelement,66
351,datepublished,89
353,author,98
354,geo,124
355,disambiguatingdescription,141
356,address,145
358,material,179
359,breadcrumb,197


,colcount_LocalB,num_col
112,owns,40
116,founder,57
120,location,118
121,hasmap,120
123,paymentaccepted,150
125,citystatezip,198
126,streetaddress,231
127,addressregion,231
128,postalcode,232
129,addresslocality,236


,colcount_Event,num_col
36,duration,77
37,doortime,111
38,typicalagerange,135
39,performers,159
43,eventattendancemode,1568
44,eventstatus,2011
46,performer,5086
47,organizer,5538
49,enddate,6798
50,location,6875


In [87]:
display(colbook)
colbook.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsbook.csv')
colmusica.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsMusicAlbum.csv')
coltv.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsTVEpisode.csv')
colRestaurant.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsRestaurant.csv')
colPerson.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsPerson.csv')
colHotel.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsHotel.csv')
colPlace.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsPlace.csv')
colProduct.to_csv('../../src/data/data/CSV_files/stats_num_tables_columnsProduct.csv')

display(colmusica)
display(coltv)
display(colRestaurant)
display(colHotel)
display(colPerson)
display(colPlace)
display(colProduct)

NameError: name 'colbook' is not defined